In [ ]:
# pip install pyspark
from pyspark import SparkContext, RDD

DIR_PATH = "Sparks_dataset/Sparks_dataset"
DIR_CONTEXT = DIR_PATH + "/Context"
DIR_DATA_FRAME = DIR_PATH + "/Data Frame"
DIR_SPARK_PANDAS = DIR_PATH + "/Spark Pandas"

# Exercise 1 - Basic Order Analysis
Tasks:

1. Load the dataset into Spark using SparkContext and textFile().
2. Count the total number of orders.
3. Calculate the total revenue from all orders.
4. Find the number of unique customers.

In [ ]:
# 1. Load the dataset into Spark using SparkContext and textFile().
sc = SparkContext("local", "ST2CBD-Lab2-OrderAnalysis")

# Load the dataset
salesdata_rdd = sc.textFile(f"{DIR_CONTEXT}/salesdata.csv.txt")

In [ ]:
# 2. Count the total number of orders.
# Because the file has 2 header lines, we need to remove them before counting
header = salesdata_rdd.take(2)

# We remove the header lines from the RDD
salesdata_rdd = salesdata_rdd.filter(lambda line: line != header[0] and line != header[1])

# Count the total number of orders
total_orders = salesdata_rdd.count()

total_orders

In the CSV file, we have 2 header lines. In order to count the total number of orders, we need to remove the header lines from the RDD.

Total number of orders: 10

In [ ]:
# 3. Calculate the total revenue from all orders.
# Index of OrderAmount is 2

sales_revenues = salesdata_rdd.map(lambda line: float(line.split(",")[2]))

total_revenue = sales_revenues.sum()

total_orders

Total orders = 10

In [ ]:
# 4. Find the number of unique customers.
# Index of CustomerID is 1

unique_customers = salesdata_rdd.map(lambda line: line.split(",")[1]).distinct()

total_unique_customers = unique_customers.count()

total_unique_customers

Total unique customers = 6

In [ ]:
sc.stop()

# Exercise 2 - Department Salary Stats
Tasks:

1. Load employee data and skip header.
2. Find total number of employees.
3. Calculate average salary per department.
4. Find department with highest total salary.

In [ ]:
# 1. Load employee data and skip header.
sc = SparkContext("local", "ST2CBD-Lab2-DepartmentSalaryStats")

# Load the dataset
salesdata_rdd = sc.textFile(f"{DIR_CONTEXT}/2 salary.csv")

# Skip header
header = salesdata_rdd.take(1)

salesdata_rdd = salesdata_rdd.filter(lambda line: line != header[0])

salesdata_rdd.take(1)

Result should not be the header and be the first line: ['E101,Alice,Sales,50000']

In [ ]:
# 2. Find total number of employees.

total_employees = salesdata_rdd.count()

total_employees

Total employees = 8

In [ ]:
# 3. Calculate average salary per department.

average_salary_per_department = salesdata_rdd.map(lambda line: (line.split(",")[2], float(line.split(",")[3]))) \
    .groupByKey() \
    .mapValues(lambda salaries: sum(salaries) / len(salaries)).collect()

average_salary_per_department

Departements average salaries:
- Sales: 54000
- Marketing: 56500
- IT: 69000

In [ ]:
# 4. Find department with the highest total salary.

highest_salary_department = salesdata_rdd.map(lambda line: (line.split(",")[2], float(line.split(",")[3]))) \
    .groupByKey() \
    .mapValues(lambda salaries: sum(salaries)) \
    .sortBy(lambda x: x[1], ascending=False) \
    .first()

highest_salary_department

In [ ]:
sc.stop()

# Exercise 3: Web Traffic Summary
Dataset: web_logs.txt

Tasks:
1. Load the data and parse it into fields.
2. Count total number of visits per URL.
3. Find unique users per URL.
4. Determine peak hour for traffic.

In [ ]:
# 1. Load the data and parse it into fields.

sc = SparkContext("local", "ST2CBD-Lab2-WebTrafficSummary")
web_logs_rdd = sc.textFile(f"{DIR_CONTEXT}/3 web_logs.txt")

# Skip header
header = web_logs_rdd.take(1)
web_logs_rdd = web_logs_rdd.filter(lambda line: line != header[0])

# Parse the data into fields
web_logs_rdd = web_logs_rdd.map(lambda line: line.split(","))

In [ ]:
# 2. Count total number of visits per URL.

url_visits = web_logs_rdd.map(lambda fields: (fields[1], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()

url_visits

We take the URL and the number of visits -> Reduce by key to count visits (line = 1 visit).

Result: [('/home', 6), ('/product', 2), ('/contact', 1), ('/about', 1)]

In [ ]:
# 3. Find unique users per URL.

unique_users_per_url = web_logs_rdd.map(lambda fields: (fields[1], fields[0])) \
    .distinct() \
    .map(lambda x: (x[0], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()

unique_users_per_url

First we take the URL and the user ID -> Remove duplicates -> Reduce by key to count unique users.

Result: [('/home', 4), ('/product', 2), ('/contact', 1), ('/about', 1)]

In [ ]:
# 4. Determine peak hour for traffic.
from datetime import datetime

hour_counts = web_logs_rdd.map(lambda fields: (datetime.strptime(fields[2], "%Y-%m-%d %H:%M:%S").hour, 1)) \
    .reduceByKey(lambda a, b: a + b)

peak_hour = hour_counts.sortBy(lambda x: x[1], ascending=False).first()

peak_hour

We take the field index 2 (Timestamp) and convert it to a datetime object -> Get the hour -> Reduce by key to count visits per hour.

After that we sort by the number of visits and take the first one.

Result: (13, 2) -> 13:00 w/ 2 visits

In [ ]:
sc.stop()

# Exercise 4: Product Rating Analysis
Tasks: Dataset: product_reviews.csv
1. Load and parse the review data.
2. Calculate the average rating per product.
3. Count number of reviews per product.
4. Find products with all 5-star reviews.

In [25]:
def parse_rdd_csv(file_path, header_lines=1) -> RDD[str]:
    rdd = sc.textFile(file_path)
    header = rdd.take(header_lines)
    rdd = rdd.filter(lambda line: line not in header)

    rdd = rdd.map(lambda line: line.split(","))
    return rdd

Created a function for parsing input files. It takes the file path and the number of header lines to skip (header). It returns an RDD with the parsed data.

In [30]:
# 1. Load and parse the review data.
sc = SparkContext("local", "ST2CBD-Lab2-ProductRatingAnalysis")
production_reviews_rdd = parse_rdd_csv(f"{DIR_CONTEXT}/4 product_reviews.csv")

[['R001', 'P001', '5'],
 ['R002', 'P002', '4'],
 ['R003', 'P001', '5'],
 ['R004', 'P003', '3'],
 ['R005', 'P002', '4'],
 ['R006', 'P003', '5'],
 ['R007', 'P001', '5'],
 ['R008', 'P004', '5'],
 ['R009', 'P004', '5'],
 ['R010', 'P005', '4']]

In [ ]:
# 2. Calculate the average rating per product
average_rating_per_product = production_reviews_rdd.map(lambda fields: (fields[1], float(fields[2]))) \
    .groupByKey() \
    .mapValues(lambda ratings: sum(ratings) / len(ratings)) \
    .collect()

average_rating_per_product

We take the second (product) and third field (score) -> Group by product -> Calculate the average score.

In [31]:
# 3. Count number of reviews per product

reviews_per_product = production_reviews_rdd.map(lambda fields: (fields[1], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()

reviews_per_product

[('P001', 3), ('P002', 2), ('P003', 2), ('P004', 2), ('P005', 1)]

We take the second field (product) and count the number of reviews (each line correspond to one product review).

In [ ]:
# 4. Find products with all 5-star reviews.

five_star_products = production_reviews_rdd.map(lambda fields: (fields[1], float(fields[2]))) \
    .groupByKey() \
    .filter(lambda x: all(rating == 5.0 for rating in x[1])) \
    .map(lambda x: x[0]) \
    .collect()

five_star_products

In [29]:
sc.stop()